In [1]:
""" We got the accuracy of ~90% on our MNIST dataset with our vanilla model, which is
unacceptable. The dataset is basically solved and state of the art models reach accuracies above
99%. You can use whatever loss functions, optimizers, even models that you want, as long as
your model is built in TensorFlow. You can save your code in the file q2.py. In the comments,
explain what you decided to do, instruction on how to run your code, and report your results.
I’m happy if you can reach 97%.
"""
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
mnist = input_data.read_data_sets('./data/mnist', one_hot=True) 

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz


In [43]:
tf.reset_default_graph()
# Step 2: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# there are 10 classes for each image, corresponding to digits 0 - 9. 
# Features are of the type float, and labels are of the type int
X =  tf.placeholder(dtype= tf.float32, shape=(None, 784), name ='input_X')
Y = tf.placeholder(dtype=tf.int16, shape=(None, 10), name = 'output_Y')
keep_prob = tf.placeholder(dtype=tf.float32, shape=[], name = 'keep_prob')
# Step 3: create weights and bias
# weights and biases are initialized to 0
# shape of w depends on the dimension of X and Y so that Y = X * w + b
# shape of b depends on Y
#w = tf.Variable(tf.zeros([784,10]), name = 'weight')
# layer-1
w1 = tf.Variable(tf.random_normal(shape=[784, 448], stddev=0.01), name='weights_1')  ## initialize all to zero or nomal  doesnt matter here
b1 = tf.Variable(tf.zeros([1, 448]), name ='bias_1')
# layer-2
w2 = tf.Variable(tf.random_normal(shape=[448, 192], stddev=0.01), name='weights_2')  ## initialize all to zero or nomal  doesnt matter here
b2 = tf.Variable(tf.zeros([1, 192]), name ='bias_2')
# layer-2
w3 = tf.Variable(tf.random_normal(shape=[192, 96], stddev=0.01), name='weights_3')  ## initialize all to zero or nomal  doesnt matter here
b3 = tf.Variable(tf.zeros([1, 96]), name ='bias_3')

# output-layer
w = tf.Variable(tf.random_normal(shape=[96, 10], stddev=0.01), name='weights')  ## initialize all to zero or nomal  doesnt matter here
b = tf.Variable(tf.zeros([1, 10]), name ='bias')



# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
# to get the probability distribution of possible label of the image
# DO NOT DO SOFTMAX HERE
X1 = tf.nn.relu(tf.matmul(X, w1) + b1)
X1_keep = tf.nn.dropout(X1, keep_prob)

X2 = tf.nn.relu(tf.matmul(X1_keep, w2) + b2)
X2_keep = tf.nn.dropout(X2, keep_prob)

X3 = tf.nn.relu(tf.matmul(X2_keep, w3) + b3)
X3_keep = tf.nn.dropout(X3, keep_prob)

logits = tf.matmul(X3_keep, w) + b

# Step 5: define loss function
# use cross entropy loss of the real labels with the softmax of logits
# use the method:
# tf.nn.softmax_cross_entropy_with_logits(logits, Y)
# then use tf.reduce_mean to get the mean loss of the batch
Y_predicted = tf.argmax(logits, axis= 1)
Y_true = tf.argmax(Y, axis= 1)
correct_preds = tf.reduce_sum(tf.cast(tf.equal(Y_predicted, Y_true), tf.float32))

xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
loss = tf.reduce_mean(xentropy) # computes the mean over all the examples in the batch

# Step 6: define training op
# using gradient descent to minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate= 0.001).minimize(loss)

In [44]:
batch_size = 512
n_epochs = 200

In [45]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())	
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0

        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            # TO-DO: run optimizer + fetch loss_batch
            _, loss_batch = sess.run([optimizer, loss], feed_dict={X:X_batch, Y: Y_batch, keep_prob: 0.5})
            
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!') # should be around 0.35 after 25 epochs
    
    
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        correct_preds_batch = sess.run(correct_preds, feed_dict={X: X_batch, Y:Y_batch, keep_prob:1})
        #accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y:Y_batch})  
        #This way, accuracy_bath will be a list
        
        #print(type(accuracy_batch))
        total_correct_preds += correct_preds_batch
    
    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 0: 1.1543755848831105
Average loss epoch 1: 0.48081847607532396
Average loss epoch 2: 0.3271671505731957
Average loss epoch 3: 0.2524201255813937
Average loss epoch 4: 0.21051144063751273
Average loss epoch 5: 0.1799540798240733
Average loss epoch 6: 0.1631539305933168
Average loss epoch 7: 0.14381615441536236
Average loss epoch 8: 0.13449134621943268
Average loss epoch 9: 0.12073376377887815
Average loss epoch 10: 0.1097979530274311
Average loss epoch 11: 0.11008031760281492
Average loss epoch 12: 0.10197792626986994
Average loss epoch 13: 0.0942220403421148
Average loss epoch 14: 0.0907497001794454
Average loss epoch 15: 0.08573850481031096
Average loss epoch 16: 0.08111251448498707
Average loss epoch 17: 0.07691869888210966
Average loss epoch 18: 0.07091789633453449
Average loss epoch 19: 0.07417704614105626
Average loss epoch 20: 0.06803944988924766
Average loss epoch 21: 0.06785782692936536
Average loss epoch 22: 0.06190382381117789
Average loss epoch 23: 0.0593

Average loss epoch 187: 0.01566367223423636
Average loss epoch 188: 0.014671729533785136
Average loss epoch 189: 0.01626886651311592
Average loss epoch 190: 0.014927109596798214
Average loss epoch 191: 0.014519936301762883
Average loss epoch 192: 0.016163808913797
Average loss epoch 193: 0.015991117465788515
Average loss epoch 194: 0.017239719248489938
Average loss epoch 195: 0.01667369432773476
Average loss epoch 196: 0.01806180602131046
Average loss epoch 197: 0.01715420129291156
Average loss epoch 198: 0.01574820525091246
Average loss epoch 199: 0.016504533573654348
Total time: 301.74692010879517 seconds
Optimization Finished!
Accuracy 0.9578


### full-layer neural network accuracy below 96%
```Average loss epoch 199: 0.016504533573654348
Total time: 301.74692010879517 seconds
Optimization Finished!
Accuracy 0.9578```

## Let's try CNN

In [7]:
tf.reset_default_graph()
# Step 2: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# there are 10 classes for each image, corresponding to digits 0 - 9. 
# Features are of the type float, and labels are of the type int
X =  tf.placeholder(dtype= tf.float32, shape=(None, 784), name ='input_X')
Y = tf.placeholder(dtype=tf.int16, shape=(None, 10), name = 'output_Y')
isTrain = tf.placeholder(dtype=tf.bool, shape=[], name = 'Train_enable')

imag = tf.reshape(X, shape=(-1, 28, 28, 1))
conv1 = tf.layers.conv2d(
      inputs=imag,
      filters=32,
      kernel_size=[5, 5],
      kernel_initializer = tf.contrib.layers.xavier_initializer(),
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      kernel_initializer = tf.contrib.layers.xavier_initializer(),
      padding="same",
      activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
conv3 = tf.layers.conv2d(
      inputs=pool2,
      filters=128,
      kernel_size=[5, 5],
      kernel_initializer = tf.contrib.layers.xavier_initializer(),
      padding="same",
      activation=tf.nn.relu)
pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

  # Dense Layer
pool3_flat = tf.reshape(pool3, [-1, 3 * 3 * 128])
dense = tf.layers.dense(inputs=pool3_flat, units=1024, activation=tf.nn.relu)

dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training= isTrain)

logits = tf.layers.dense(inputs=dropout, units=10)

Y_predicted = tf.argmax(logits, axis= 1)
Y_true = tf.argmax(Y, axis= 1)
correct_preds = tf.reduce_sum(tf.cast(tf.equal(Y_predicted, Y_true), tf.float32))

xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
loss = tf.reduce_mean(xentropy) # computes the mean over all the examples in the batch

# Step 6: define training op
# using gradient descent to minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate= 0.0001).minimize(loss)

In [8]:
batch_size = 256
n_epochs = 100

In [9]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())	
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0

        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            # TO-DO: run optimizer + fetch loss_batch
            _, loss_batch = sess.run([optimizer, loss], feed_dict={X:X_batch, Y: Y_batch, isTrain: True})
            
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!') # should be around 0.35 after 25 epochs
    
    
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        correct_preds_batch = sess.run(correct_preds, feed_dict={X: X_batch, Y:Y_batch, isTrain:False})
        #accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y:Y_batch})  
        #This way, accuracy_bath will be a list
        
        #print(type(accuracy_batch))
        total_correct_preds += correct_preds_batch
    
    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 0: 0.7313583569827481
Average loss epoch 1: 0.1595021419784176
Average loss epoch 2: 0.10928969036474406
Average loss epoch 3: 0.08368257516733954
Average loss epoch 4: 0.07370360853655317
Average loss epoch 5: 0.0591595581271381
Average loss epoch 6: 0.053877470040014974
Average loss epoch 7: 0.048385685544774355
Average loss epoch 8: 0.043193596502261185
Average loss epoch 9: 0.03943725789366203
Average loss epoch 10: 0.03516551532356121
Average loss epoch 11: 0.0326293211465127
Average loss epoch 12: 0.028863875278071546
Average loss epoch 13: 0.026935691939814835
Average loss epoch 14: 0.024451664268152318
Average loss epoch 15: 0.02328879704533545
Average loss epoch 16: 0.021098840532380495
Average loss epoch 17: 0.019984008404904993
Average loss epoch 18: 0.01819322693431489
Average loss epoch 19: 0.01604412627301971
Average loss epoch 20: 0.015333258368457867
Average loss epoch 21: 0.01485249760264757
Average loss epoch 22: 0.013210741714378558
Average loss ep

### bingo!!!
```Average loss epoch 99: 0.0004988794280995884
Total time: 4403.048403263092 seconds
Optimization Finished!
Accuracy 0.9914
```

## Task 3: Build a logistic regression model to predict whether someone has coronary heart disease

In [22]:
import pandas as pd
df_heart = pd.read_csv(r'./data/heart.csv')
df_heart['famhist_Present'] = (df_heart['famhist'] == 'Present').astype(int)
df_heart['famhist_Absent'] = (df_heart['famhist'] != 'Present').astype(int)
df_heart = df_heart.drop('famhist', axis= 1)

In [23]:
df_heart.head()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd,famhist_Present,famhist_Absent
0,160,12.00,5.73,23.11,49,25.30,97.20,52,1,1,0
1,144,0.01,4.41,28.61,55,28.87,2.06,63,1,0,1
2,118,0.08,3.48,32.28,52,29.14,3.81,46,0,1,0
3,170,7.50,6.41,38.03,51,31.99,24.26,58,1,1,0
4,134,13.60,3.50,27.78,60,25.99,57.34,49,1,1,0


In [93]:
avg = df_heart[['sbp', 'tobacco', 'ldl','adiposity', 'typea', 'obesity', 'alcohol', 'age']].mean()
std = df_heart[['sbp', 'tobacco', 'ldl','adiposity', 'typea', 'obesity', 'alcohol', 'age']].std()

In [94]:
data_raw = df_heart[['sbp', 'tobacco', 'ldl','adiposity', 'typea', 'obesity', 'alcohol', 'age']]

In [95]:
df_heart[['sbp', 'tobacco', 'ldl','adiposity', 'typea', 'obesity', 'alcohol', 'age']] =  (data_raw- avg) / std 

In [96]:
df_heart.head()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd,famhist_Present,famhist_Absent
0,1.057417,1.821099,0.477894,-0.295183,-0.418017,-0.176594,3.274189,0.628654,1,1,0
1,0.276789,-0.789382,-0.159507,0.411694,0.193134,0.670646,-0.612081,1.381617,1,0,1
2,-0.991731,-0.774141,-0.608585,0.883374,-0.112441,0.734723,-0.540597,0.217947,0,1,0
3,1.545310,0.841352,0.806252,1.622382,-0.214300,1.411091,0.294742,1.039361,1,1,0
4,-0.211103,2.169453,-0.598928,0.305020,0.702427,-0.012842,1.645991,0.423301,1,1,0


In [97]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df_heart, test_size=0.2, random_state=42)

In [98]:
train_set.head()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd,famhist_Present,famhist_Absent
411,1.350153,-0.617382,0.429606,1.388470,-0.316158,2.058981,0.480192,1.176264,0,0,1
265,-0.991731,-0.765432,-1.342562,-0.655048,-1.640320,-1.432029,-0.597376,-1.698685,0,0,1
57,-0.601417,0.318821,-0.859683,0.140510,0.193134,-0.124384,-0.192165,-0.329662,1,0,1
199,0.471946,-0.791559,-0.038787,-0.130674,-0.316158,-0.186087,-0.531611,-1.082625,0,0,1
175,-0.796574,0.079327,0.241283,0.319158,-0.825451,0.112939,-0.696228,1.244715,1,1,0


In [99]:
feature_train = train_set.drop('chd', axis=1).copy().as_matrix()
feature_test = test_set.drop('chd', axis=1).copy().as_matrix()
labels_train = train_set['chd'].copy().as_matrix()
labels_test = test_set['chd'].copy().as_matrix()

In [100]:
def batch_generator(features,labels, batch_size):
    num_batch = len(features) // batch_size
    for i in range(num_batch):
        yield features[i*batch_size: (i+1)*batch_size], labels[i*batch_size: (i+1)*batch_size]
    yield features[num_batch * batch_size:], labels[num_batch * batch_size:]

In [112]:
tf.reset_default_graph()

features = tf.placeholder(dtype=tf.float32, shape=[None, 10], name = 'features')
labels = tf.placeholder(dtype= tf.float32, shape=[None], name ='labels')
isTraining = tf.placeholder(dtype=tf.bool, shape =[], name='Training_enable')

batch_norm1= tf.layers.batch_normalization(features, training= isTraining)
fc1 = tf.layers.dense(inputs=batch_norm1, units=64, activation=tf.nn.relu)

batch_norm2= tf.layers.batch_normalization(fc1, training= isTraining)
fc2 = tf.layers.dense(inputs=batch_norm2, units=32, activation=tf.nn.relu)

batch_norm3= tf.layers.batch_normalization(fc2, training= isTraining)
logits = tf.squeeze(tf.layers.dense(inputs=batch_norm3, units=1, activation=None))
sigmoid = tf.sigmoid(logits)

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logits, labels = labels))

labels_predicted = tf.cast(tf.greater(sigmoid, 0.5), tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(labels_predicted, labels), tf.float32))

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)

In [113]:
batch_size = 32
n_epochs = 100

In [114]:
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())	
    for i in range(n_epochs):
        m =0
        for feature, label in batch_generator(feature_train,labels_train, batch_size): 
            _, loss_batch, accuracy_batch = sess.run([optimizer, loss, accuracy], feed_dict={features:feature, labels: label, isTraining: True})
            m +=1
            print('ephoch:{0},batch:{1}, accuracy:{2}' .format(i, m, accuracy_batch))
            
    accuracy_test = sess.run(accuracy, feed_dict={features:feature_test, labels: labels_test, isTraining: True})
    print('accuracy test:', accuracy_test)

ephoch:0,batch:1, accuracy:0.53125
ephoch:0,batch:2, accuracy:0.53125
ephoch:0,batch:3, accuracy:0.5
ephoch:0,batch:4, accuracy:0.4375
ephoch:0,batch:5, accuracy:0.5625
ephoch:0,batch:6, accuracy:0.40625
ephoch:0,batch:7, accuracy:0.46875
ephoch:0,batch:8, accuracy:0.40625
ephoch:0,batch:9, accuracy:0.46875
ephoch:0,batch:10, accuracy:0.4375
ephoch:0,batch:11, accuracy:0.34375
ephoch:0,batch:12, accuracy:0.47058823704719543
ephoch:1,batch:1, accuracy:0.5625
ephoch:1,batch:2, accuracy:0.59375
ephoch:1,batch:3, accuracy:0.5
ephoch:1,batch:4, accuracy:0.4375
ephoch:1,batch:5, accuracy:0.59375
ephoch:1,batch:6, accuracy:0.4375
ephoch:1,batch:7, accuracy:0.5
ephoch:1,batch:8, accuracy:0.4375
ephoch:1,batch:9, accuracy:0.4375
ephoch:1,batch:10, accuracy:0.4375
ephoch:1,batch:11, accuracy:0.34375
ephoch:1,batch:12, accuracy:0.47058823704719543
ephoch:2,batch:1, accuracy:0.5625
ephoch:2,batch:2, accuracy:0.5625
ephoch:2,batch:3, accuracy:0.4375
ephoch:2,batch:4, accuracy:0.5
ephoch:2,batch:5, 

ephoch:21,batch:12, accuracy:0.6470588445663452
ephoch:22,batch:1, accuracy:0.75
ephoch:22,batch:2, accuracy:0.625
ephoch:22,batch:3, accuracy:0.6875
ephoch:22,batch:4, accuracy:0.78125
ephoch:22,batch:5, accuracy:0.6875
ephoch:22,batch:6, accuracy:0.6875
ephoch:22,batch:7, accuracy:0.59375
ephoch:22,batch:8, accuracy:0.625
ephoch:22,batch:9, accuracy:0.6875
ephoch:22,batch:10, accuracy:0.625
ephoch:22,batch:11, accuracy:0.65625
ephoch:22,batch:12, accuracy:0.6470588445663452
ephoch:23,batch:1, accuracy:0.75
ephoch:23,batch:2, accuracy:0.625
ephoch:23,batch:3, accuracy:0.6875
ephoch:23,batch:4, accuracy:0.78125
ephoch:23,batch:5, accuracy:0.6875
ephoch:23,batch:6, accuracy:0.6875
ephoch:23,batch:7, accuracy:0.625
ephoch:23,batch:8, accuracy:0.6875
ephoch:23,batch:9, accuracy:0.71875
ephoch:23,batch:10, accuracy:0.65625
ephoch:23,batch:11, accuracy:0.65625
ephoch:23,batch:12, accuracy:0.6470588445663452
ephoch:24,batch:1, accuracy:0.75
ephoch:24,batch:2, accuracy:0.625
ephoch:24,batch:3

ephoch:43,batch:5, accuracy:0.75
ephoch:43,batch:6, accuracy:0.75
ephoch:43,batch:7, accuracy:0.6875
ephoch:43,batch:8, accuracy:0.78125
ephoch:43,batch:9, accuracy:0.71875
ephoch:43,batch:10, accuracy:0.75
ephoch:43,batch:11, accuracy:0.78125
ephoch:43,batch:12, accuracy:0.7647058963775635
ephoch:44,batch:1, accuracy:0.78125
ephoch:44,batch:2, accuracy:0.75
ephoch:44,batch:3, accuracy:0.75
ephoch:44,batch:4, accuracy:0.8125
ephoch:44,batch:5, accuracy:0.75
ephoch:44,batch:6, accuracy:0.75
ephoch:44,batch:7, accuracy:0.71875
ephoch:44,batch:8, accuracy:0.78125
ephoch:44,batch:9, accuracy:0.71875
ephoch:44,batch:10, accuracy:0.75
ephoch:44,batch:11, accuracy:0.78125
ephoch:44,batch:12, accuracy:0.7647058963775635
ephoch:45,batch:1, accuracy:0.78125
ephoch:45,batch:2, accuracy:0.75
ephoch:45,batch:3, accuracy:0.75
ephoch:45,batch:4, accuracy:0.8125
ephoch:45,batch:5, accuracy:0.75
ephoch:45,batch:6, accuracy:0.75
ephoch:45,batch:7, accuracy:0.75
ephoch:45,batch:8, accuracy:0.78125
ephoch

ephoch:62,batch:9, accuracy:0.75
ephoch:62,batch:10, accuracy:0.78125
ephoch:62,batch:11, accuracy:0.8125
ephoch:62,batch:12, accuracy:0.7647058963775635
ephoch:63,batch:1, accuracy:0.8125
ephoch:63,batch:2, accuracy:0.90625
ephoch:63,batch:3, accuracy:0.75
ephoch:63,batch:4, accuracy:0.90625
ephoch:63,batch:5, accuracy:0.78125
ephoch:63,batch:6, accuracy:0.75
ephoch:63,batch:7, accuracy:0.78125
ephoch:63,batch:8, accuracy:0.8125
ephoch:63,batch:9, accuracy:0.75
ephoch:63,batch:10, accuracy:0.8125
ephoch:63,batch:11, accuracy:0.8125
ephoch:63,batch:12, accuracy:0.8235294222831726
ephoch:64,batch:1, accuracy:0.8125
ephoch:64,batch:2, accuracy:0.90625
ephoch:64,batch:3, accuracy:0.75
ephoch:64,batch:4, accuracy:0.90625
ephoch:64,batch:5, accuracy:0.78125
ephoch:64,batch:6, accuracy:0.75
ephoch:64,batch:7, accuracy:0.78125
ephoch:64,batch:8, accuracy:0.8125
ephoch:64,batch:9, accuracy:0.75
ephoch:64,batch:10, accuracy:0.8125
ephoch:64,batch:11, accuracy:0.8125
ephoch:64,batch:12, accuracy

ephoch:82,batch:8, accuracy:0.84375
ephoch:82,batch:9, accuracy:0.75
ephoch:82,batch:10, accuracy:0.84375
ephoch:82,batch:11, accuracy:0.84375
ephoch:82,batch:12, accuracy:0.9411764740943909
ephoch:83,batch:1, accuracy:0.8125
ephoch:83,batch:2, accuracy:0.9375
ephoch:83,batch:3, accuracy:0.8125
ephoch:83,batch:4, accuracy:0.90625
ephoch:83,batch:5, accuracy:0.8125
ephoch:83,batch:6, accuracy:0.78125
ephoch:83,batch:7, accuracy:0.8125
ephoch:83,batch:8, accuracy:0.84375
ephoch:83,batch:9, accuracy:0.75
ephoch:83,batch:10, accuracy:0.875
ephoch:83,batch:11, accuracy:0.84375
ephoch:83,batch:12, accuracy:0.9411764740943909
ephoch:84,batch:1, accuracy:0.8125
ephoch:84,batch:2, accuracy:0.9375
ephoch:84,batch:3, accuracy:0.8125
ephoch:84,batch:4, accuracy:0.90625
ephoch:84,batch:5, accuracy:0.8125
ephoch:84,batch:6, accuracy:0.78125
ephoch:84,batch:7, accuracy:0.8125
ephoch:84,batch:8, accuracy:0.875
ephoch:84,batch:9, accuracy:0.75
ephoch:84,batch:10, accuracy:0.875
ephoch:84,batch:11, accu

 ### sandard scaler does not Help